# Exercise 3 Solution: Two-Strain Model (R Version)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ngozzi/tech-transfer-epdemix/blob/main/sessions/session-4/solutions/r-colab/exercise_3_two_strains.ipynb)

Model the emergence of a more transmissible variant using a multi-strain SIR model, using R via the `reticulate` package.

In [ ]:
!pip install epydemix
%load_ext rpy2.ipython

In [ ]:
%%R
if (!require("reticulate", quietly = TRUE)) {
  install.packages("reticulate")
}
library(reticulate)
use_python("/usr/bin/python3", required = TRUE)

## Model Structure

**Compartments:** S, I1, I2, R1, R2

**Transitions:**
- S + I1 → 2I1 (rate β)
- S + I2 → 2I2 (rate β·ψ, where ψ > 1)
- R1 + I2 → I2 + R2 (rate β·ψ·γ, partial cross-immunity)
- I1 → R1 (rate μ)
- I2 → R2 (rate μ)

## Task 1: Implement the Two-Strain Model

In [ ]:
%%R
# Import epydemix
epydemix <- import("epydemix")
EpiModel <- epydemix$EpiModel
builtins <- import_builtins()
np <- import("numpy")
viz <- import("epydemix.visualization")
plot_quantiles <- viz$plot_quantiles

# Function to create a two-strain model
create_two_strain_model <- function(psi = 1.5, gamma_val = 0.4, beta = 0.20, mu = 0.1) {
  model <- EpiModel(name = "Two-Strain SIR", compartments = c("S", "I1", "I2", "R1", "R2"))
  
  # S + I1 -> 2I1
  params_S_I1 <- builtins$tuple(list("beta", "I1"))
  model$add_transition(source = "S", target = "I1", kind = "mediated", params = params_S_I1)
  
  # S + I2 -> 2I2 (more transmissible)
  params_S_I2 <- builtins$tuple(list("beta*psi", "I2"))
  model$add_transition(source = "S", target = "I2", kind = "mediated", params = params_S_I2)
  
  # R1 + I2 -> I2 + R2 (reinfection)
  params_R1_I2 <- builtins$tuple(list("beta*psi*gamma", "I2"))
  model$add_transition(source = "R1", target = "I2", kind = "mediated", params = params_R1_I2)
  
  # I1 -> R1
  model$add_transition(source = "I1", target = "R1", kind = "spontaneous", params = "mu1")
  
  # I2 -> R2
  model$add_transition(source = "I2", target = "R2", kind = "spontaneous", params = "mu2")
  
  # Set parameters
  model$add_parameter("beta", beta)
  model$add_parameter("psi", psi)
  model$add_parameter("gamma", gamma_val)
  model$add_parameter("mu1", mu)
  model$add_parameter("mu2", mu)
  
  return(model)
}

# Create the model
model <- create_two_strain_model(psi = 1.5, gamma_val = 0.4)
model

## Task 2: Delay Strain 2 Emergence

We use `override_parameter` to set ψ=0 and μ2=0 for the first 30 days, effectively turning off strain 2.

In [ ]:
%%R
# Override parameters to delay strain 2 emergence
model$override_parameter(
  start_date = "2025-01-01",
  end_date = "2025-01-31",
  parameter_name = "psi",
  value = 0.0
)

model$override_parameter(
  start_date = "2025-01-01",
  end_date = "2025-01-31",
  parameter_name = "mu2",
  value = 0.0
)

## Task 3: Simulate and Visualize

In [ ]:
%%R
# Get default population Nk
Nk_val <- py_to_r(model$population$Nk)

# Initial conditions
initial_conditions <- reticulate::dict(
  S = np$array(as.integer(Nk_val - 20)),
  I1 = np$array(as.integer(10)),
  I2 = np$array(as.integer(10)),
  R1 = np$array(as.integer(0)),
  R2 = np$array(as.integer(0))
)

# Run simulations
results <- model$run_simulations(
  start_date = "2025-01-01",
  end_date = "2025-07-31",
  Nsim = 50L,
  initial_conditions_dict = initial_conditions
)

In [ ]:
%%R
# Plot both strains
df_quantiles <- results$get_quantiles_compartments()

sns <- import("seaborn")
colors <- py_to_r(sns$color_palette("Dark2"))

ax <- plot_quantiles(df_quantiles, columns = c("I1_total"),
                     colors = colors[[1]], labels = list("Strain 1 (original)"))
ax <- plot_quantiles(df_quantiles, columns = c("I2_total"),
                     colors = colors[[2]], labels = list("Strain 2 (more transmissible)"), ax = ax)

# Mark emergence date
ax$axvline("2025-02-01", color = "red", linestyle = "--", alpha = 0.5, label = "Strain 2 emerges")

ax$set_title("Two-Strain Dynamics: Emergence of a More Transmissible Variant")
ax$set_ylabel("Infected")
ax$legend()
ax

## Task 4: Experiments

### Experiment A: Effect of Emergence Timing

In [ ]:
%%R
run_with_emergence_day <- function(emergence_day) {
  m <- create_two_strain_model(psi = 1.5, gamma_val = 0.4)
  
  emergence_date <- as.character(as.Date("2025-01-01") + emergence_day)
  
  m$override_parameter(
    start_date = "2025-01-01", end_date = emergence_date,
    parameter_name = "psi", value = 0.0
  )
  m$override_parameter(
    start_date = "2025-01-01", end_date = emergence_date,
    parameter_name = "mu2", value = 0.0
  )
  
  Nk_val <- py_to_r(m$population$Nk)
  ic <- reticulate::dict(
    S = np$array(as.integer(Nk_val - 20)),
    I1 = np$array(as.integer(10)),
    I2 = np$array(as.integer(10)),
    R1 = np$array(as.integer(0)),
    R2 = np$array(as.integer(0))
  )
  
  res <- m$run_simulations(
    start_date = "2025-01-01", end_date = "2025-07-31",
    Nsim = 50L, initial_conditions_dict = ic
  )
  
  list(results = res, emergence_date = emergence_date)
}

# Run with different emergence times
res_early <- run_with_emergence_day(15)
res_mid <- run_with_emergence_day(30)
res_late <- run_with_emergence_day(45)

In [ ]:
%%R
plt <- import("matplotlib.pyplot")

# Plot early emergence
df <- res_early$results$get_quantiles_compartments()
ax <- plot_quantiles(df, columns = c("I1_total"), colors = colors[[1]], labels = list("Strain 1"))
ax <- plot_quantiles(df, columns = c("I2_total"), colors = colors[[2]], labels = list("Strain 2"), ax = ax)
ax$axvline(res_early$emergence_date, color = "red", linestyle = "--", alpha = 0.5)
ax$set_title("Early emergence (day 15)")
ax$legend()
ax

In [ ]:
%%R
# Plot late emergence
df <- res_late$results$get_quantiles_compartments()
ax <- plot_quantiles(df, columns = c("I1_total"), colors = colors[[1]], labels = list("Strain 1"))
ax <- plot_quantiles(df, columns = c("I2_total"), colors = colors[[2]], labels = list("Strain 2"), ax = ax)
ax$axvline(res_late$emergence_date, color = "red", linestyle = "--", alpha = 0.5)
ax$set_title("Late emergence (day 45)")
ax$legend()
ax

### Experiment B: Effect of Transmissibility Advantage

In [ ]:
%%R
run_with_psi <- function(psi_value) {
  m <- create_two_strain_model(psi = psi_value, gamma_val = 0.4)
  
  m$override_parameter(
    start_date = "2025-01-01", end_date = "2025-01-31",
    parameter_name = "psi", value = 0.0
  )
  m$override_parameter(
    start_date = "2025-01-01", end_date = "2025-01-31",
    parameter_name = "mu2", value = 0.0
  )
  
  Nk_val <- py_to_r(m$population$Nk)
  ic <- reticulate::dict(
    S = np$array(as.integer(Nk_val - 20)),
    I1 = np$array(as.integer(10)),
    I2 = np$array(as.integer(10)),
    R1 = np$array(as.integer(0)),
    R2 = np$array(as.integer(0))
  )
  
  m$run_simulations(
    start_date = "2025-01-01", end_date = "2025-07-31",
    Nsim = 50L, initial_conditions_dict = ic
  )
}

results_psi_low <- run_with_psi(1.2)
results_psi_mid <- run_with_psi(1.5)
results_psi_high <- run_with_psi(2.0)

In [ ]:
%%R
# Plot psi = 1.2
df <- results_psi_low$get_quantiles_compartments()
ax <- plot_quantiles(df, columns = c("I1_total"), colors = colors[[1]], labels = list("Strain 1"))
ax <- plot_quantiles(df, columns = c("I2_total"), colors = colors[[2]], labels = list("Strain 2"), ax = ax)
ax$axvline("2025-02-01", color = "red", linestyle = "--", alpha = 0.5)
ax$set_title(expression(paste(psi, " = 1.2 (20% advantage)")))
ax$legend()
ax

In [ ]:
%%R
# Plot psi = 2.0
df <- results_psi_high$get_quantiles_compartments()
ax <- plot_quantiles(df, columns = c("I1_total"), colors = colors[[1]], labels = list("Strain 1"))
ax <- plot_quantiles(df, columns = c("I2_total"), colors = colors[[2]], labels = list("Strain 2"), ax = ax)
ax$axvline("2025-02-01", color = "red", linestyle = "--", alpha = 0.5)
ax$set_title(expression(paste(psi, " = 2.0 (100% advantage)")))
ax$legend()
ax

In [ ]:
%%R
# Compute dominance day
compute_dominance_day <- function(results) {
  traj <- results$get_stacked_compartments()
  I1_median <- apply(py_to_r(traj["I1_total"]), 2, median)
  I2_median <- apply(py_to_r(traj["I2_total"]), 2, median)
  
  crossover <- which(I2_median > I1_median & I2_median > 100)
  if (length(crossover) > 0) return(crossover[1])
  return(NA)
}

cat("Days until strain 2 dominance:\n")
cat(sprintf("  psi = 1.2: Day %s\n", compute_dominance_day(results_psi_low)))
cat(sprintf("  psi = 1.5: Day %s\n", compute_dominance_day(results_psi_mid)))
cat(sprintf("  psi = 2.0: Day %s\n", compute_dominance_day(results_psi_high)))

## Discussion

**Key observations:**

1. **Emergence timing matters:** Early emergence allows strain 2 to take over quickly before strain 1 depletes susceptibles. Late emergence means fewer susceptibles remain, but strain 2 can still spread among R1 individuals.

2. **Transmissibility advantage (ψ):** Higher ψ leads to faster strain replacement. Even modest advantages (20%) lead to eventual dominance.

3. **Cross-immunity (γ):** γ = 0.4 means R1 individuals have 60% protection against strain 2. This slows but doesn't prevent spread.

**Real-world implications:**
- Early detection of new variants is crucial for pandemic response
- Even small transmissibility advantages compound through exponential growth
- Vaccines providing cross-variant protection can slow variant emergence